In [3]:
import re
import csv
import os

In [2]:
def parse_ycsb_log(filenames):
    data_dict = {
        "timestamp": [],
        "Database": [],
        "workload_type": [],
        "Operations": [],
        "AverageLatency": [],
        "RunTime": [],
        "Throughput": [],
        "MinLatency": [],
        "MaxLatency": [],
        "95thPercentileLatency": [],
        "99thPercentileLatency": []
    }

    for filename in filenames:
        with open(filename, "r") as f:
            lines = f.readlines()
            
            workload_type = None
            database = None
            for line in lines:
                timestamp_match = re.search(r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})", line)
                if timestamp_match:
                    timestamp = timestamp_match.group(1)
                
                workload_match = re.search(r"# ycsb load (\w+)", line)
                if workload_match:
                    database = workload_match.group(1)
                    workload_type = re.search(r"# ycsb load (\w+)([a-zA-Z0-9]*?) load", line).group(2)
                
                if workload_type and database:
                    result_match = re.search(r"\[(INSERT)\], ([\w\(\)/%]+), ([\d.]+)", line)
                    if result_match:
                        metric_type = result_match.group(1)
                        metric_name = result_match.group(2)
                        metric_value = float(result_match.group(3))

                        if metric_name in data_dict.keys():
                            data_dict["timestamp"].append(timestamp)
                            data_dict["Database"].append(database)
                            data_dict["workload_type"].append(workload_type)
                            data_dict[metric_name].append(metric_value)
                            for key in data_dict.keys():
                                if key != "timestamp" and key != "Database" and key != "workload_type" and key != metric_name:
                                    data_dict[key].append(None)

    with open('ycsb_report.csv', 'w', newline='') as csvfile:
        fieldnames = data_dict.keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for i in range(len(data_dict["timestamp"])):
            row_dict = {fn: data_dict[fn][i] for fn in fieldnames}
            writer.writerow(row_dict)

